# **Übung 8a** Programmierung mit Python mit Anwendungen aus dem Maschinellen Lernen



## Aufgabe 1
Diese Übung soll ein erweitertes Bild zum Training von neuronalen Netzen liefern.
Unter anderem sollen folgende Themen angeschnitten werden:
- Batching und Datasets
- Netzwerkdesign

Als Startpunkt dient der in `sklearn` integrierte Diabetes-Datensatz. Mehr Information dazu erhalten Sie in der Ausgabe der nächsten Zelle.
Es werden auch folgende relevante Variablen erzeugt:
- Der Datensatz: `diabetes`
- Daraus erzeugte Trainingsdaten: `X_train, X_test, y_train, y_test`

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

import numpy as np

import pandas as pd

import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as Data
from torch import optim

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import seaborn as sns
import tqdm
from matplotlib import pyplot as plt

diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target
X = X.astype(np.float32)
y = y.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

USE_GPU = False

n_samples, n_features = X.shape
print(diabetes['DESCR'])

**Aufgabe 1.0 (PyTorch)** | Im Folgenden wurde der `TransformedTargetRegressor` auf das Problem angewandt. Dies soll Ihnen als Anhaltspunkt für einen *schon-ganz-guten* Regressor dienen. 

Es stellt sich die Frage: Kann das Ergebnis durch den Einsatz eines neuronalen Netzes weiter verbessert werden? Die folgenden Aufgaben sind als Guideline gedacht. Sie müssen sich nicht daran halten.

Falls gewollt oder nötig: Zur ersten Analyse von Daten eignen sich bei wenigen (1-5) Features `sns.pairplot` oder bei mehr (5-25) Features `sns.heatmap`.

```
Beispiel:
sns.pairplot(df)
sns.heatmap(df.corr(), annot=True)
```

In [ ]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import QuantileTransformer

regr_trans = TransformedTargetRegressor(
    regressor=RidgeCV(),
    transformer=QuantileTransformer(
        n_quantiles=257, output_distribution='uniform'))

regr_trans.fit(X_train, y_train)
y_pred = regr_trans.predict(X_test)

def showRes(y_pred, y_target):
  fig = px.scatter(x=y_pred, y=y_target, color=np.abs(y_pred-y_target), width=500, height=500)
  fig.add_shape(type="line",
                x0=0, 
                y0=0, 
                x1=350, 
                y1=350)
  fig.update_layout(
      xaxis_title="prediction",
      yaxis_title="actual",
      title = f"MAE: {mean_absolute_error(y_target, y_pred):.1f} - MSE: {mean_squared_error(y_target, y_pred):.1f} - R2: {r2_score(y_target, y_pred):.3f}",
      coloraxis_colorbar_title_text = 'Distance',
  )
  fig.show()
  print("MAE:", mean_absolute_error(y_target, y_pred))
  print("MSE:", mean_squared_error(y_target, y_pred))
  print("R2:", r2_score(y_target, y_pred))
  
showRes(y_pred, y_test)

**Aufgabe 1.1 (PyTorch)** | Erzeugen Sie sich einen [DataGenerator](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files). Dieser kann von einem [DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#preparing-your-data-for-training-with-dataloaders) konsumiert werden, welcher häufig verwendete Basisfunktionalität für das Training und Testing liefert.
- Dataloader(IO)
- Batching
- Shuffle
- Parallelität
- Sampling


In [ ]:
class DiabetesDataGenerator(Data.Dataset):
    def __init__(self, X, y):
        self.targets = torch.from_numpy(X.astype(np.float32))
        self.labels = torch.from_numpy(y.astype(np.float32))
        if USE_GPU:
          self.targets = self.targets.cuda()
          self.labels = self.labels.cuda()
    
    def __getitem__(self, i):
        return self.targets[i, :], self.labels[i]

    def __len__(self):
        return len(self.targets)

**Aufgabe 1.2 (PyTorch)** | Erzeugen Sie sich eine [Modell](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html#define-the-class), versuchen Sie verschiedene `Linear`-Layer, jeweils mit Aktivierungsfunktionen zu verbinden.

Als Referenz:
- 2-6 Linear-Layer
- 5-250 Features je Layer
- Aktivierungsfunktionen: Tanh, Sigmoid, ReLU, SiLU, ..

In [ ]:

# Es können komplexere, eigene Modelle geschrieben werden
class Model(nn.Module):
    def __init__(self, n_features, hidden1, hidden2):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(n_features, hidden1)
        self.linear2 = nn.Linear(hidden1, hidden2)
        self.linear3 = nn.Linear(hidden2, 1)

    def forward(self, x):
        y1 = F.silu(self.linear1(x))
        y2 = F.silu(self.linear2(y1))
        return self.linear3(y2)

# Oder wenn ausreichend, können diese aus bestehenden Modellen zusammengesetzt werden.
# Hier könnte auch die oben definierte Klasse 'Model' verwendet werden.
Model2 = lambda n_features: nn.Sequential(
    nn.Linear(n_features, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

**Aufgabe 1.3 (PyTorch)** | [Trainieren](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#full-implementation) Sie ihr Modell.
- Testen Sie Batching
- Speichern Sie auch den Test-Loss 
- Zeigen Sie den Trainingsverlauf an

In [ ]:
torch.manual_seed(2)
train_set = DiabetesDataGenerator(X_train, y_train)
test_set = DiabetesDataGenerator(X_test, y_test)
batch_size = 100
train_loader = Data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = Data.DataLoader(test_set, batch_size=batch_size, shuffle=True)
#model = Model(n_features, 150, 100)
model = Model2(n_features)
if USE_GPU:
  model = model.cuda() 
loss_fn = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=0.1)

n_epochs = 150
all_losses = []

progress_bar = tqdm.tqdm(range(n_epochs), leave=False)

for epoch in progress_bar:
    losses = []
    losses_test = []
    total = 0
    total2 = 0
    for inputs, target in train_loader:
        optimizer.zero_grad()
        y_pred = model(inputs)
        loss = loss_fn(y_pred, torch.unsqueeze(target,dim=1))

        loss.backward()
        
        optimizer.step()
        
        progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
        losses.append(loss.item())
        total += 1

    for inputs_test, target_test in test_loader:

        y_pred_test = model(inputs_test)
        loss_test = loss_fn(y_pred_test, torch.unsqueeze(target_test,dim=1))

        losses_test.append(loss_test.item())
        total2 += 1

    epoch_loss = sum(losses) / total
    epoch_loss_test = sum(losses_test) / total2

    all_losses.append([epoch, epoch_loss, epoch_loss_test])

df_loss = pd.DataFrame(np.array(all_losses), columns=['epoch', 'loss_train', 'loss_test'])

fig = px.line(df_loss, x='epoch', y=['loss_train','loss_test'], width=750, height=500, log_y=True)
fig.update_layout(
    xaxis_title="epoch",
    yaxis_title="loss",
)
fig.show()

**Aufgabe 1.4 (PyTorch)** | Erzeugen Sie sich die Ausgaben aus `Aufgabe 1.0` und vergleichen Sie die Ergebnisse. Konnten die Werte verbessert werden?

In [ ]:
y_pred_model = []
y_target_model = []
model.train(False)
for inputs, targets in test_loader:
    y_pred_model.extend(model(inputs).data.cpu().numpy())
    y_target_model.extend(targets.cpu().numpy())

y_pred_model = np.array(y_pred_model).flatten()
y_target_model = np.array(y_target_model)

# showRes(y_pred, y_test) ## TransformedTargetRegressor result
showRes(y_pred_model, y_target_model) ## model result